In [1]:
# Setup and Installation for Google Earth Engine

#!pip install earthengine-api
#!pip install geemap
#pip install rasterio geopandas matplotlib folium
#!pip install pycrs
#!pip install osmnx

In [1]:
# Imports
import ee
import os
import json
import datetime, xarray
import rasterio
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from shapely.geometry import shape

In [2]:
# Authenticate to Earth Engine (you'll need to run this once)
# Uncomment the next line if this is your first time using GEE
ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()

In [4]:
# Define 2024 date range
start_date = '2024-01-01'
end_date = '2024-12-31'

print(f"Collecting data for: {start_date} to {end_date}")

# Initialize layers dictionary for export
layers_dict = {}

In [5]:
import geemap

# Read the local shapefile using geopandas
gdf = gpd.read_file('../assets/shapefile/zc04AdminBoundaries.shp')
print(f"Loaded shapefile with {len(gdf)} features")
print(f"CRS: {gdf.crs}")

# Convert GeoDataFrame → Earth Engine FeatureCollection
roi = geemap.geopandas_to_ee(gdf)

# Create and display map
m = geemap.Map(ee_initialize=False)
m.centerObject(roi, 10)
m.addLayer(roi, {}, 'Study Area')
print("ROI added to map")

Loaded shapefile with 101 features
CRS: EPSG:32651
ROI added to map


In [6]:
# Load Sentinel-2
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Verify we have 2024 data
collection_size = s2.size().getInfo()
print(f"Sentinel-2 2024 images found: {collection_size}")

if collection_size > 0:
    # Create a cloud-free composite using median
    s2_composite = s2.median()
    
    # Define visualization parameters for true color
    vis_params_rgb = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000
    }
    
    # Add to map
    m.addLayer(s2_composite, vis_params_rgb, 'Sentinel-2 RGB (2024)')
    
    # Calculate NDVI from 2024 data
    ndvi = s2_composite.normalizedDifference(['B8', 'B4']).rename('NDVI')
    vis_params_ndvi = {
        'min': -0.2,
        'max': 0.9,
        'palette': ['blue', 'white', 'green']
    }
    m.addLayer(ndvi, vis_params_ndvi, 'NDVI (2024)')
    
    # Calculate NDBI (Built-up Index) from 2024 data
    ndbi = s2_composite.normalizedDifference(['B11', 'B8']).rename('NDBI')
    vis_params_ndbi = {
        'min': -0.3,
        'max': 0.5,
        'palette': ['blue', 'white', 'red']
    }
    m.addLayer(ndbi, vis_params_ndbi, 'NDBI (2024)')
else:
    print("WARNING: No Sentinel-2 data found for 2024")

# Store layers
layers_dict['sentinel2_composite'] = s2_composite
layers_dict['ndvi'] = ndvi
layers_dict['ndbi'] = ndbi

Sentinel-2 2024 images found: 34


In [7]:
# Load VIIRS Nighttime Lights
viirs = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Verify we have 2024 data
viirs_size = viirs.size().getInfo()
print(f"VIIRS 2024 monthly images found: {viirs_size}")

if viirs_size > 0:
    # Create a composite using median
    viirs_composite = viirs.median()
    
    # Extract the nighttime lights band
    ntl = viirs_composite.select('avg_rad')
    
    # Define visualization parameters
    vis_params_ntl = {
        'min': 0,
        'max': 50,
        'palette': ['black', 'yellow', 'orange', '#FF5349']
    }
    
    # Add to map
    m.addLayer(ntl, vis_params_ntl, 'Nighttime Lights (2024)')
else:
    print("WARNING: No VIIRS data found for 2024")

layers_dict['nighttime_lights'] = ntl

VIIRS 2024 monthly images found: 12


In [8]:
# Load MODIS Vegetation Index (16-day)
modis_vi = ee.ImageCollection('MODIS/061/MOD13A2') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Verify we have 2024 data
modis_vi_size = modis_vi.size().getInfo()
print(f"MODIS Vegetation Index 2024 images found: {modis_vi_size}")

if modis_vi_size > 0:
    # Create composite using median
    modis_vi_composite = modis_vi.median()
    
    # Extract NDVI and scale it
    modis_ndvi = modis_vi_composite.select('NDVI').multiply(0.0001)
    
    # Define visualization parameters
    vis_params_modis_ndvi = {
        'min': -0.2,
        'max': 0.9,
        'palette': ['blue', 'white', 'green']
    }
    
    # Add to map
    m.addLayer(modis_ndvi, vis_params_modis_ndvi, 'MODIS NDVI (2024)')
else:
    print("WARNING: No MODIS Vegetation Index data found for 2024")

layers_dict['modis_ndvi'] = modis_ndvi

MODIS Vegetation Index 2024 images found: 23


In [9]:
# Load CHIRPS daily precipitation
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Verify we have 2024 data
chirps_size = chirps.size().getInfo()
print(f"CHIRPS 2024 daily precipitation images found: {chirps_size}")

if chirps_size > 0:
    # Create a composite of cumulative precipitation for 2024
    precip_sum = chirps.sum().select('precipitation')
    
    # Optional: Get actual min/max values from your data
    stats = precip_sum.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=5000,
        maxPixels=1e9
    ).getInfo()
    print(f"Precipitation range: {stats}")
    
    # Define visualization parameters
    vis_params_precip = {
        'min': 0,
        'max': 3000,  # Typical annual rainfall range (adjust for your region)
        'palette': ['white', 'lightblue', 'blue', 'darkblue', 'purple']
        # Alternative: ['#FFFFFF', '#B0E0E6', '#4682B4', '#0000CD', '#4B0082']
    }
    
    # Add to map
    m.addLayer(precip_sum, vis_params_precip, 'Cumulative Precipitation (2024)')
else:
    print("WARNING: No CHIRPS precipitation data found for 2024")

layers_dict['precipitation'] = precip_sum

CHIRPS 2024 daily precipitation images found: 365
Precipitation range: {'precipitation_max': 2912.7143862247467, 'precipitation_min': 1415.0246028974652}


In [10]:
# Including for reference as it's essential for topographic analysis
print("NOTE: SRTM is static data from 2000 - no 2024 equivalent available")

srtm = ee.Image('USGS/SRTMGL1_003')

# Calculate slope and aspect
elevation = srtm.select('elevation').clip(roi)
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

# Define visualization parameters
vis_params_elevation = {
    'min': 0,
    'max': 500,
    'palette': ['#375e2b', '#5a8643', '#94bc6e', '#c9e08d', '#efeebf', '#e0c494', '#c08b65', '#a05c3e', '#833025', '#681a17', '#4f1313']
}

vis_params_slope = {
    'min': 0,
    'max': 30,
    'palette': ['#f7fcfd', '#e0ecf4', '#bfd3e6', '#9ebcda', '#8c96c6', '#8c6bb1', '#88419d', '#6e016b']
}

# Add to map
m.addLayer(elevation, vis_params_elevation, 'Elevation (SRTM - Static)')
m.addLayer(slope, vis_params_slope, 'Slope (SRTM - Static)')

layers_dict['elevation'] = elevation
layers_dict['slope'] = slope

NOTE: SRTM is static data from 2000 - no 2024 equivalent available


In [11]:
# Load Landsat 8/9 Collection 2 - STRICTLY 2024
landsat = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Verify we have 2024 data
landsat_size = landsat.size().getInfo()
print(f"Landsat 8/9 2024 images found: {landsat_size}")

if landsat_size > 0:
    # Create composite
    landsat_composite = landsat.median()
    
    # FIXED: Calculate Surface Temperature correctly for Collection 2
    # Collection 2 surface temperature conversion:
    # 1. Apply correct scale factor (0.00341802)
    # 2. Add offset (149.0) - gives temp in Kelvin
    # 3. Convert from Kelvin to Celsius (subtract 273.15)
    st = landsat_composite.select('ST_B10') \
        .multiply(0.00341802) \
        .add(149.0) \
        .subtract(273.15)
    
    # Define visualization parameters for Celsius temperature
    vis_params_st = {
        'min': 20,
        'max': 40,  # Typical temperature range in Celsius
        'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red']
    }
    
    # Add to map with clipping
    m.addLayer(st.clip(roi), vis_params_st, 'Surface Temperature (°C)')
else:
    print("WARNING: No Landsat 8/9 data found for 2024")

layers_dict['surface_temp'] = st

Landsat 8/9 2024 images found: 5


In [12]:
# Gridded Population of the World
worldpop_path = '../assets/phl_pop_2024_CN_100m_R2025A_v1.tif'

with rasterio.open(worldpop_path) as src:
    print(f"GeoTIFF Info:")
    print(f"CRS: {src.crs}")
    print(f"Resolution: {src.res}")
    print(f"Bounds: {src.bounds}")
    print(f"Shape: {src.shape}")

worldpop_asset_id = 'projects/ee-zc-povertymapping/assets/phl_pop_2024_CN_100m_R2025A_v1'

# Load the WorldPop GeoTIFF from Earth Engine assets
worldpop_image = ee.Image(worldpop_asset_id)

# Clip to your study area (optional, but recommended)
worldpop_clipped = worldpop_image.clip(roi)

# Define visualization parameters
vis_params_pop = {
    'min': 0,
    'max': 1000,  # Adjust based on your data's max value
    'palette': ['white', 'yellow', 'orange', 'red', 'darkred']
}

# Add to map
m.addLayer(worldpop_clipped, vis_params_pop, 'WorldPop Population Density (2024)')

layers_dict['population'] = worldpop_clipped

GeoTIFF Info:
CRS: EPSG:4326
Resolution: (0.00083333333, 0.00083333333)
Bounds: BoundingBox(left=116.92833214561999, bottom=4.58750031764999, right=126.60499877358, top=21.12166691817999)
Shape: (19841, 11612)


In [13]:
# add osm layers
# Load and add POIs layer
try:
    # Load POIs GeoJSON
    pois_gdf = gpd.read_file("../assets/POIs-OSM.geojson")
    print(f"✓ Loaded {len(pois_gdf)} POIs from OSM")
    
    # Add POIs to map with styling
    poi_style = {
        'color': 'red',
        'radius': 3,
        'fillColor': 'orange',
        'fillOpacity': 0.7,
        'weight': 1
    }
    
    m.add_gdf(pois_gdf, layer_name="OSM POIs", style=poi_style)
    print("✓ Added POIs layer to map")
    
except Exception as e:
    print(f"✗ Error loading POIs: {str(e)}")

# Load and add Roads layer
try:
    # Load Roads GeoJSON
    roads_gdf = gpd.read_file("../assets/roads-OSM.geojson")
    print(f"✓ Loaded {len(roads_gdf)} road features from OSM")
    
    # Add Roads to map with styling
    road_style = {
        'color': 'blue',
        'weight': 2,
        'opacity': 0.8
    }
    
    m.add_gdf(roads_gdf, layer_name="OSM Roads", style=road_style)
    print("✓ Added Roads layer to map")
    
except Exception as e:
    print(f"✗ Error loading Roads: {str(e)}")

✓ Loaded 1822 POIs from OSM
✓ Added POIs layer to map
✓ Loaded 12640 road features from OSM
✓ Added Roads layer to map


In [14]:
print("=== DEBUGGING GEOJSON LOADING ===")

# Check if files exist
pois_path = "../assets/POIs-OSM.geojson"
roads_path = "../assets/roads-OSM.geojson"

print(f"POIs file exists: {os.path.exists(pois_path)}")
print(f"Roads file exists: {os.path.exists(roads_path)}")

# Load and inspect the data
try:
    pois_gdf = gpd.read_file(pois_path)
    print(f"✓ Loaded {len(pois_gdf)} POIs")
    print(f"  Geometry types: {pois_gdf.geometry.type.value_counts().to_dict()}")
    print(f"  Columns: {list(pois_gdf.columns)}")
    print(f"  CRS: {pois_gdf.crs}")
    
    # Check if geometries are valid
    valid_geoms = pois_gdf.geometry.is_valid.sum()
    print(f"  Valid geometries: {valid_geoms}/{len(pois_gdf)}")
    
except Exception as e:
    print(f"✗ Error loading POIs: {str(e)}")
    pois_gdf = None

try:
    roads_gdf = gpd.read_file(roads_path)
    print(f"\n✓ Loaded {len(roads_gdf)} roads")
    print(f"  Geometry types: {roads_gdf.geometry.type.value_counts().to_dict()}")
    print(f"  Columns: {list(roads_gdf.columns)}")
    print(f"  CRS: {roads_gdf.crs}")
    
    # Check if geometries are valid
    valid_geoms = roads_gdf.geometry.is_valid.sum()
    print(f"  Valid geometries: {valid_geoms}/{len(roads_gdf)}")
    
except Exception as e:
    print(f"✗ Error loading roads: {str(e)}")
    roads_gdf = None

=== DEBUGGING GEOJSON LOADING ===
POIs file exists: True
Roads file exists: True
✓ Loaded 1822 POIs
  Geometry types: {'Point': 1822}
  Columns: ['id', '@id', 'FIXME', 'abbr', 'access', 'accommodation', 'addr2:street', 'addr3:street', 'addr:barangay', 'addr:building', 'addr:city', 'addr:country', 'addr:district', 'addr:floor', 'addr:full', 'addr:hamlet', 'addr:housename', 'addr:housenumber', 'addr:milestone', 'addr:neighbourhood', 'addr:place', 'addr:postcode', 'addr:province', 'addr:purok', 'addr:quarter', 'addr:street', 'addr:street:corner', 'addr:subdistrict', 'addr:suburb', 'addr:unit', 'addr:village', 'admin_level', 'air_conditioning', 'alt_name', 'alt_name:ur', 'amenity', 'artwork_type', 'atm', 'backrest', 'barrier', 'beds', 'bench', 'bin', 'boundary', 'branch', 'brand', 'brand:en', 'brand:facebook', 'brand:instagram', 'brand:ja', 'brand:twitter', 'brand:website', 'brand:wikidata', 'brand:wikipedia', 'brand:wikipedia:es', 'building', 'building:levels', 'building:part', 'building:

In [39]:
print("=== REDUCING GEOJSON FILE SIZE (UPDATED FILTERS) ===")

# Convert ROI to WGS84 for matching
gdf_wgs84 = gdf.to_crs('EPSG:4326')
roi_geom = gdf_wgs84.unary_union  # Merge all features into one geometry

# --- POIs (unchanged but kept for completeness) ---
if pois_gdf is not None:
    print(f"Original POIs: {len(pois_gdf)} features")
    if pois_gdf.crs != 'EPSG:4326':
        pois_gdf = pois_gdf.to_crs('EPSG:4326')
    pois_filtered = pois_gdf[pois_gdf.intersects(roi_geom.buffer(0.01))]
    print(f"After spatial filter: {len(pois_filtered)} features")
    if len(pois_filtered) == 0:
        print("⚠️ WARNING: No POIs in ROI! Keeping all POIs from original file")
        pois_filtered = pois_gdf
    essential_cols = ['geometry']
    if 'amenity' in pois_filtered.columns:
        essential_cols.append('amenity')
    if 'name' in pois_filtered.columns:
        essential_cols.append('name')
    pois_simple = pois_filtered[essential_cols].copy()
    pois_simple['geometry'] = pois_simple.geometry.simplify(0.0001)  # ~10m tolerance
    print(f"Final POIs: {len(pois_simple)} features")
    pois_simple.to_file("../assets/POIs-simple.geojson", driver='GeoJSON')
    print("✓ Saved simplified POIs")

# --- Roads (UPDATED) ---
if roads_gdf is not None:
    print(f"\nOriginal roads: {len(roads_gdf)} features")

    # Ensure same CRS
    if roads_gdf.crs != 'EPSG:4326':
        roads_gdf = roads_gdf.to_crs('EPSG:4326')

    # Filter to ROI intersection (with small buffer)
    roads_filtered = roads_gdf[roads_gdf.intersects(roi_geom.buffer(0.01))]
    print(f"After spatial filter: {len(roads_filtered)} features")

    if len(roads_filtered) == 0:
        print("⚠️ WARNING: No roads in ROI! Keeping all roads from original file")
        roads_filtered = roads_gdf

    # Keep only essential columns (preserve 'highway' and 'name' if present)
    essential_cols = ['geometry']
    if 'highway' in roads_filtered.columns:
        essential_cols.append('highway')
    if 'name' in roads_filtered.columns:
        essential_cols.append('name')

    roads_simple = roads_filtered[essential_cols].copy()

    # Simplify geometries (tolerance adjustable). Use a moderate tolerance to reduce vertex count but keep topology.
    simplify_tolerance = 0.0005  # ~50m; tune between 0.0001 and 0.001
    roads_simple['geometry'] = roads_simple.geometry.simplify(simplify_tolerance)
    print(f"Simplified geometries with tolerance={simplify_tolerance}")

    # Recommended highway types to keep (includes local streets to avoid accessibility gaps)
    keep_highways = [
        'motorway', 'trunk', 'primary', 'secondary', 'tertiary',
        'trunk_link', 'primary_link', 'secondary_link',
        # local access (highly recommended)
        'residential', 'service', 'unclassified', 'track',
        # pedestrian / last-mile (optional)
        'path', 'footway', 'pedestrian', 'steps'
    ]

    if 'highway' in roads_simple.columns:
        before_count = len(roads_simple)
        roads_simple = roads_simple[roads_simple['highway'].isin(keep_highways)]
        after_count = len(roads_simple)
        print(f"Filtered by highway types: kept {after_count} / {before_count} features")
    else:
        print("No 'highway' column found on roads_simple; keeping all filtered features")

    print(f"Final roads: {len(roads_simple)} features")

    # Save simplified version (this is the full filtered set, not sampled)
    out_path = "../assets/roads-simple.geojson"
    try:
        roads_simple.to_file(out_path, driver='GeoJSON')
        print(f"✓ Saved simplified roads to {out_path}")
    except Exception as e:
        print("✗ Saving roads-simple.geojson failed:", e)
        # Fallback: try saving a small sample for debugging
        try:
            sample_n = min(2000, len(roads_simple))
            roads_sample = roads_simple.sample(n=sample_n, random_state=42)
            sample_path = "../assets/roads-simple-sample.geojson"
            roads_sample.to_file(sample_path, driver='GeoJSON')
            print(f"✓ Saved sample ({sample_n}) to {sample_path} for debugging")
        except Exception as e2:
            print("✗ Fallback sample save also failed:", e2)

=== REDUCING GEOJSON FILE SIZE (UPDATED FILTERS) ===
Original POIs: 1822 features
After spatial filter: 1822 features
Final POIs: 1822 features
✓ Saved simplified POIs

Original roads: 12640 features
After spatial filter: 12640 features
Simplified geometries with tolerance=0.0005
Filtered by highway types: kept 12037 / 12640 features
Final roads: 12037 features
✓ Saved simplified roads to ../assets/roads-simple.geojson


In [40]:
# Convert simplified data to EE
print("=== CONVERTING SIMPLIFIED DATA TO EE ===")

pois_ee = None
roads_ee = None

# Load simplified POIs
try:
    pois_simple = gpd.read_file("../assets/POIs-simple.geojson")
    if pois_simple.crs != 'EPSG:4326':
        pois_simple = pois_simple.to_crs('EPSG:4326')
    
    # Convert to EE in smaller chunks if still large
    if len(pois_simple) > 1000:
        # Take a sample for testing
        pois_sample = pois_simple.sample(n=1000, random_state=42)
        pois_ee = geemap.gdf_to_ee(pois_sample)
        print(f"✓ POIs converted to EE (sample): {pois_ee.size().getInfo()} features")
    else:
        pois_ee = geemap.gdf_to_ee(pois_simple)
        print(f"✓ POIs converted to EE: {pois_ee.size().getInfo()} features")
        
except Exception as e:
    print(f"✗ Error converting simplified POIs: {str(e)}")

# Load simplified roads
try:
    roads_simple = gpd.read_file("../assets/roads-simple.geojson")
    if roads_simple.crs != 'EPSG:4326':
        roads_simple = roads_simple.to_crs('EPSG:4326')
    
    # Convert to EE in smaller chunks if still large
    if len(roads_simple) > 500:
        # Take a sample for testing
        roads_sample = roads_simple.sample(n=500, random_state=42)
        roads_ee = geemap.gdf_to_ee(roads_sample)
        print(f"✓ Roads converted to EE (sample): {roads_ee.size().getInfo()} features")
    else:
        roads_ee = geemap.gdf_to_ee(roads_simple)
        print(f"✓ Roads converted to EE: {roads_ee.size().getInfo()} features")
        
except Exception as e:
    print(f"✗ Error converting simplified roads: {str(e)}")

=== CONVERTING SIMPLIFIED DATA TO EE ===
✓ POIs converted to EE (sample): 1000 features
✓ Roads converted to EE (sample): 500 features


In [41]:
print("\n=== VERIFYING POI AND ROAD DATA ===")

# Check if conversions succeeded
if pois_ee is not None:
    try:
        poi_count = pois_ee.size().getInfo()
        print(f"✓ POIs in EE: {poi_count} features")
        
        # Check bounds
        poi_bounds = pois_ee.geometry().bounds().getInfo()
        print(f"  POI bounds: {poi_bounds}")
    except Exception as e:
        print(f"✗ Error checking POIs: {str(e)}")
        pois_ee = None
else:
    print("✗ POIs not loaded")

if roads_ee is not None:
    try:
        road_count = roads_ee.size().getInfo()
        print(f"✓ Roads in EE: {road_count} features")
        
        # Check bounds
        road_bounds = roads_ee.geometry().bounds().getInfo()
        print(f"  Road bounds: {road_bounds}")
    except Exception as e:
        print(f"✗ Error checking roads: {str(e)}")
        roads_ee = None
else:
    print("✗ Roads not loaded")


=== VERIFYING POI AND ROAD DATA ===
✓ POIs in EE: 1000 features
  POI bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[121.902181, 6.8676274], [122.3534548, 6.8676274], [122.3534548, 7.4544062], [121.902181, 7.4544062], [121.902181, 6.8676274]]]}
✓ Roads in EE: 500 features
  Road bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[121.89840270000003, 6.888643199999975], [122.3661518, 6.888643199999975], [122.3661518, 7.466424400000024], [121.89840270000003, 7.466424400000024], [121.89840270000003, 6.888643199999975]]]}


In [20]:
print("\n=== VERIFYING POI AND ROAD DATA ===")

# Check if conversions succeeded
if pois_ee is not None:
    try:
        poi_count = pois_ee.size().getInfo()
        print(f"✓ POIs in EE: {poi_count} features")
        
        # Check bounds
        poi_bounds = pois_ee.geometry().bounds().getInfo()
        print(f"  POI bounds: {poi_bounds}")
    except Exception as e:
        print(f"✗ Error checking POIs: {str(e)}")
        pois_ee = None
else:
    print("✗ POIs not loaded")

if roads_ee is not None:
    try:
        road_count = roads_ee.size().getInfo()
        print(f"✓ Roads in EE: {road_count} features")
        
        # Check bounds
        road_bounds = roads_ee.geometry().bounds().getInfo()
        print(f"  Road bounds: {road_bounds}")
    except Exception as e:
        print(f"✗ Error checking roads: {str(e)}")
        roads_ee = None
else:
    print("✗ Roads not loaded")


=== VERIFYING POI AND ROAD DATA ===
✓ POIs in EE: 1000 features
  POI bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[121.902181, 6.8676274], [122.3534548, 6.8676274], [122.3534548, 7.4544062], [121.902181, 7.4544062], [121.902181, 6.8676274]]]}
✓ Roads in EE: 500 features
  Road bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[121.90203350000002, 6.898031699999976], [122.35082570000002, 6.898031699999976], [122.35082570000002, 7.495544600000026], [121.90203350000002, 7.495544600000026], [121.90203350000002, 6.898031699999976]]]}


In [42]:
print("\nCreating POI and road accessibility layers (robust rasterized approach)...")

# Parameters
METER_CRS = 'EPSG:3857'
RASTER_SCALE = 30  # meters per pixel for rasterization; adjust if needed

def make_accessibility_from_fc(fc, max_dist_m, layer_name, scale=RASTER_SCALE, crs=METER_CRS):
    """Rasterize a FeatureCollection to a binary image in a meter CRS, compute euclidean distance (m),
       and return an accessibility image (max_dist - distance) clamped to [0, max_dist]."""
    if fc is None:
        print(f"✗ {layer_name}: FeatureCollection is None")
        return None
    try:
        # Ensure fc is a FeatureCollection (if passed as a geometry/list, convert)
        fc = ee.FeatureCollection(fc)

        # Compute distance (meters) directly from the FeatureCollection.
        # The returned image gives, for each pixel, distance (in meters) to nearest geometry in fc.
        dist = fc.distance(max_dist_m)  # searchRadius in meters

        # Accessibility score: closer => larger value (max_dist - distance)
        access = ee.Image.constant(max_dist_m).subtract(dist).clamp(0, max_dist_m)

        # IMPORTANT: Unmask to 0 so reduceRegion always sees pixels within ROI (prevents nulls)
        access = access.unmask(0).rename(layer_name)

        # Optionally reproject (visualization/export will use scale anyway). Clip to ROI.
        proj = ee.Projection(crs).atScale(int(scale))
        access = access.reproject(proj).clip(roi)

        return access
    except Exception as e:
        print(f"✗ Error creating {layer_name}: {e}")
        return None

# Create POI accessibility (5000 m)
poi_access = make_accessibility_from_fc(pois_ee, 5000, 'poi_accessibility', scale=50)
if poi_access is not None:
    layers_dict['poi_accessibility'] = poi_access
    poi_viz = {'min': 0, 'max': 5000, 'palette': ['red', 'yellow', 'green']}
    m.addLayer(poi_access, poi_viz, "POI Accessibility")
    print("✓ Added POI accessibility layer")
else:
    print("✗ POI accessibility not created")

# Create Road accessibility (2000 m)
road_access = make_accessibility_from_fc(roads_ee, 2000, 'road_accessibility', scale=50)
if road_access is not None:
    layers_dict['road_accessibility'] = road_access
    road_viz = {'min': 0, 'max': 2000, 'palette': ['red', 'orange', 'green']}
    m.addLayer(road_access, road_viz, "Road Accessibility")
    print("✓ Added Road accessibility layer")
else:
    print("✗ Road accessibility not created")

print(f"\nUpdated layers_dict has {len([k for k,v in layers_dict.items() if v is not None])} layers")


Creating POI and road accessibility layers (robust rasterized approach)...
✓ Added POI accessibility layer
✓ Added Road accessibility layer

Updated layers_dict has 12 layers


In [43]:
# Create a grid using a simpler approach that avoids transform errors

# Verify we have data to work with
print(f"\n=== GRID CREATION SUMMARY ===")
print(f"Available layers: {list(layers_dict.keys())}")
if len(layers_dict) == 0:
    print("⚠️ WARNING: No layers available for grid sampling!")
else:
    print(f"✓ Ready to sample {len(layers_dict)} layers")
    
# We'll use degrees but approximate 1km size

# Calculate degrees for approximately 1km (varies by latitude)
lat = 7.155877217195312  # ROI centroid latitude
# Approximate conversion: 1 degree latitude = ~111.32 km globally
# 1 degree longitude = ~111.32 * cos(latitude) km
lat_km_degree = 1/111.32  # degrees per km for latitude
lon_km_degree = 1/(111.32 * np.cos(np.radians(lat)))  # degrees per km for longitude

print(f"At latitude {lat}:")
print(f"1km ≈ {lat_km_degree:.6f} degrees latitude")
print(f"1km ≈ {lon_km_degree:.6f} degrees longitude")

# Create grid using ee.FeatureCollection
def create_grid_wgs84(roi, lon_step, lat_step):
    """Create a grid using WGS84 coordinates"""
    bounds = roi.geometry().bounds()
    coords = ee.List(bounds.coordinates().get(0))
    
    x_min = ee.Number(ee.List(coords.get(0)).get(0))
    y_min = ee.Number(ee.List(coords.get(0)).get(1))
    x_max = ee.Number(ee.List(coords.get(2)).get(0))
    y_max = ee.Number(ee.List(coords.get(2)).get(1))
    
    # Create sequences with approximately 1km spacing
    x_seq = ee.List.sequence(x_min, x_max, lon_step)
    y_seq = ee.List.sequence(y_min, y_max, lat_step)
    
    # Create grid using nested mapping
    grid_features = y_seq.map(lambda y: 
        x_seq.map(lambda x:
            ee.Feature(ee.Geometry.Rectangle([
                x, y, ee.Number(x).add(lon_step), ee.Number(y).add(lat_step)
            ])).set({
                'x_idx': x_seq.indexOf(x),
                'y_idx': y_seq.indexOf(y),
                'grid_id': ee.String(x_seq.indexOf(x)).cat('_').cat(ee.String(y_seq.indexOf(y)))
            })
        )
    ).flatten()
    
    # Convert to feature collection
    grid = ee.FeatureCollection(grid_features)
    
    # Filter to ROI
    return grid.filter(ee.Filter.intersects('.geo', roi.geometry()))

# Create the grid with ~1km spacing using our calculated degrees
grid = create_grid_wgs84(roi, lon_km_degree, lat_km_degree)

# Add to map with styling
viz = {
    'color': 'FF0000',
    'width': 1.5,
    'fillColor': '00000000'
}

m.centerObject(roi, 10)
m.addLayer(grid.style(**viz), {}, "~1km² Grid")

# Verify the grid cell count
grid_count = grid.size().getInfo()
print(f"Created {grid_count} grid cells of approximately 1km²")


=== GRID CREATION SUMMARY ===
Available layers: ['sentinel2_composite', 'ndvi', 'ndbi', 'nighttime_lights', 'modis_ndvi', 'precipitation', 'elevation', 'slope', 'surface_temp', 'population', 'poi_accessibility', 'road_accessibility']
✓ Ready to sample 12 layers
At latitude 7.155877217195312:
1km ≈ 0.008983 degrees latitude
1km ≈ 0.009054 degrees longitude
Created 1724 grid cells of approximately 1km²


In [44]:
# Extract data for each grid cell with better error handling
print("\n=== PREPARING DATA EXTRACTION (FIXED BAND EXTRACTION) ===")

def extract_grid_stats(feature):
    """Extract statistics for a single grid cell"""
    # Intersect each grid cell with ROI so we only reduce over valid pixels
    geom = ee.Geometry(feature.geometry()).intersection(roi.geometry(), ee.ErrorMargin(1))
    stats = {}

    # Define expected band names for each layer
    band_names = {
        'sentinel2_composite': 'B4',  # or use first band
        'ndvi': 'NDVI',
        'ndbi': 'NDBI',
        'nighttime_lights': 'avg_rad',
        'modis_ndvi': 'NDVI',
        'precipitation': 'precipitation',
        'elevation': 'elevation',
        'slope': 'slope',  # ← This is what's missing!
        'surface_temp': 'ST_B10',
        'population': 'b1',
        'poi_accessibility': 'poi_accessibility',
        'road_accessibility': 'road_accessibility'
    }

    # Extract stats for each layer
    for layer_name, image in layers_dict.items():
        if image is not None:
            try:
                # Compute mean
                result = image.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=geom,
                    scale=100,
                    maxPixels=1e9,
                    bestEffort=True,
                    tileScale=4
                )

                # Get the correct band value
                band_name = band_names.get(layer_name)
                if band_name:
                    value = result.get(band_name)
                else:
                    # Fallback: get first available band
                    result_dict = ee.Dictionary(result)
                    keys = result_dict.keys()
                    value = ee.Algorithms.If(
                        keys.size().gt(0),
                        result_dict.get(keys.get(0)),
                        None
                    )

                stats[layer_name] = value

            except Exception as e:
                stats[layer_name] = None

    return feature.set(stats)

# Apply extraction
grid_with_data = grid.map(extract_grid_stats)

print("✓ Data extraction prepared (now includes slope!)")

# Preview with better diagnostics
print("\nPreviewing 3 sample cells...")
preview = grid_with_data.limit(3).getInfo()

for i, feat in enumerate(preview['features']):
    props = feat['properties']
    print(f"\n--- Cell {i} (ID: {props.get('grid_id', 'N/A')}) ---")
    
    # Show topographic metrics
    for key in ['elevation', 'slope']:
        if key in props:
            val = props.get(key)
            print(f"  {key}: {val}")

    # Show accessibility metrics
    access_keys = [k for k in props.keys() if 'access' in k]
    for key in access_keys:
        val = props.get(key)
        print(f"  {key}: {val}")

    # Show other key metrics
    for key in ['ndvi', 'population', 'nighttime_lights']:
        if key in props:
            val = props.get(key)
            print(f"  {key}: {val}")


=== PREPARING DATA EXTRACTION (FIXED BAND EXTRACTION) ===
✓ Data extraction prepared (now includes slope!)

Previewing 3 sample cells...

--- Cell 0 (ID: 17_0) ---
  elevation: 3.5724928366762194
  slope: 1.5861139496236114
  poi_accessibility: 4108.87564297547
  road_accessibility: 0
  ndvi: 0.45853711712624806
  population: 10.486939696998862
  nighttime_lights: 0.32038387517905537

--- Cell 1 (ID: 18_0) ---
  elevation: 3.7262435821900515
  slope: 1.4509754278364373
  poi_accessibility: 4329.324143690158
  road_accessibility: 0
  ndvi: 0.5068588239037553
  population: 18.177850200918314
  nighttime_lights: 0.3008437545630438

--- Cell 2 (ID: 19_0) ---
  elevation: 4.495290537603245
  slope: 0.7951531045235832
  poi_accessibility: 4396.407413208137
  road_accessibility: 0
  ndvi: 0.6533539926450227
  population: 17.751657502353197
  nighttime_lights: 0.3341360292929816


In [45]:
# Export grid data to CSV for analysis
print("\n=== EXPORTING GRID DATA ===")

# Prepare export task
export_task = ee.batch.Export.table.toDrive(
    collection=grid_with_data,
    description='zc04_grid_data_2024_2',
    fileFormat='CSV',
    folder='EarthEngineExports'
)

# Start the export
export_task.start()
print(f"✓ Export task started: {export_task.id}")
print("Check Google Drive folder 'EarthEngineExports' for results")


=== EXPORTING GRID DATA ===



KeyboardInterrupt

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001CA2FDCDD10>>
Traceback (most recent call last):
  File "C:\Users\Admin\miniconda3\envs\povertymapping\Lib\site-packages\ipykernel\ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [37]:
# Export grid data to CSV for analysis with parallel tasks
print("\n=== EXPORTING GRID DATA (PARALLEL) ===")

# Get total grid count
total_count = grid_with_data.size().getInfo()
print(f"Total grid cells: {total_count}")

# Calculate split sizes
chunk_size = int(np.ceil(total_count / 3))
print(f"Chunk size: {chunk_size} cells per task")

# Create 3 parallel export tasks
export_tasks = []

for i in range(3):
    start_idx = i * chunk_size
    
    # Create subset of grid_with_data
    grid_subset = grid_with_data.toList(chunk_size, start_idx)
    grid_subset_fc = ee.FeatureCollection(grid_subset)
    
    # Prepare export task
    task = ee.batch.Export.table.toDrive(
        collection=grid_subset_fc,
        description=f'zc04_grid_data_2024_part{i+1}',
        fileFormat='CSV',
        folder='EarthEngineExports'
    )
    
    # Start the export
    task.start()
    export_tasks.append(task)
    print(f"✓ Export task {i+1} started: {task.id}")
    print(f"  Processing cells {start_idx} to {start_idx + chunk_size - 1}")

print(f"\n✓ All {len(export_tasks)} parallel export tasks started")
print("Check Google Drive folder 'EarthEngineExports' for results")
print("Files will be named: zc04_grid_data_2024_part1.csv, part2.csv, part3.csv")


=== EXPORTING GRID DATA (PARALLEL) ===
Total grid cells: 1724
Chunk size: 575 cells per task
✓ Export task 1 started: AGJ3SATP53TJDK2GLGGJB76A
  Processing cells 0 to 574
✓ Export task 2 started: FHIINY63X5L5NE2DJLFSZHMW
  Processing cells 575 to 1149
✓ Export task 3 started: P6D6GKO3EYF27Z7BNNMBQSMY
  Processing cells 1150 to 1724

✓ All 3 parallel export tasks started
Check Google Drive folder 'EarthEngineExports' for results
Files will be named: zc04_grid_data_2024_part1.csv, part2.csv, part3.csv


In [ ]:
# Merge the 3 parts
df1 = pd.read_csv('../googleEarthExports/zc04_grid_data_2024_part1.csv')
df2 = pd.read_csv('../googleEarthExports/zc04_grid_data_2024_part2.csv')
df3 = pd.read_csv('../googleEarthExports/zc04_grid_data_2024_part3.csv')

# Combine into single dataframe
df_complete = pd.concat([df1, df2, df3], ignore_index=True)
df_complete.to_csv('zc04_grid_data_2024_complete.csv', index=False)

In [46]:
# Run this in a new cell to check progress
status = export_task.status()
print(f"State: {status['state']}")
print(f"Progress: {status.get('progress', 'N/A')}")
print(f"Started: {status.get('start_timestamp_ms', 'N/A')}")
print(f"Updated: {status.get('update_timestamp_ms', 'N/A')}")

State: State.UNSUBMITTED
Progress: N/A
Started: N/A
Updated: N/A


In [47]:
print("\n=== EXPORTING GRID DATA (10 PARALLEL TASKS) ===")

try:
    total_count = int(grid_with_data.size().getInfo())
except Exception as e:
    print(f"✗ Failed to get collection size: {e}")
    total_count = 0

print(f"Total grid cells: {total_count}")

if total_count == 0:
    print("⚠️ No features to export or failed to retrieve count.")
else:
    n_tasks = 10
    chunk_size = int(np.ceil(total_count / n_tasks))
    print(f"Splitting into {n_tasks} tasks, chunk size (approx): {chunk_size} cells per task")

    export_tasks = []
    for i in range(n_tasks):
        start_idx = i * chunk_size
        remaining = max(0, total_count - start_idx)
        if remaining == 0:
            break
        this_count = min(chunk_size, remaining)

        subset_fc = ee.FeatureCollection(grid_with_data.toList(this_count, start_idx))

        task = ee.batch.Export.table.toDrive(
            collection=subset_fc,
            description=f'zc04_grid_data_2024_part{i+1}',
            fileFormat='CSV',
            folder='EarthEngineExports',
            fileNamePrefix=f'zc04_grid_data_2024_part{i+1}'
        )
        try:
            task.start()
            export_tasks.append(task)
            end_idx = start_idx + this_count - 1
            print(f"✓ Export task {i+1} started: {task.id} (cells {start_idx} to {end_idx})")
        except Exception as e:
            print(f"✗ Failed to start task {i+1}: {e}")

    print(f"\n✓ Started {len(export_tasks)} export tasks. Monitor with task.status() or in the GEE Tasks tab.")


=== EXPORTING GRID DATA (10 PARALLEL TASKS) ===
Total grid cells: 1724
Splitting into 10 tasks, chunk size (approx): 173 cells per task
✓ Export task 1 started: SEHF3XCXFCZ22T2MLRA7E3KW (cells 0 to 172)
✓ Export task 2 started: T7ZKUTQMWGPABF42RAYAKQPD (cells 173 to 345)
✓ Export task 3 started: E5RGMCJNYL4J6ZQHDLCHRVWH (cells 346 to 518)
✓ Export task 4 started: QKVCGZ6TY6RJDIQT2AGB5I2I (cells 519 to 691)
✓ Export task 5 started: 7CAAQ45DIPHAEN3PQRAUBSC7 (cells 692 to 864)
✓ Export task 6 started: NAKVHNP6IRTVG6MJ5SRVZ4RW (cells 865 to 1037)
✓ Export task 7 started: 4HZQRXKDTBFK7STKRA7WOBDP (cells 1038 to 1210)
✓ Export task 8 started: KTOZM452LM756LGAFKFJZCHB (cells 1211 to 1383)
✓ Export task 9 started: 5SELQ5DJOPKTNPVOK2MSRE25 (cells 1384 to 1556)
✓ Export task 10 started: LGE5AD22ZK3QQWJ5NMCQM3DW (cells 1557 to 1723)

✓ Started 10 export tasks. Monitor with task.status() or in the GEE Tasks tab.
